In [10]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, pipeline
import re
import numpy as np
from collections import Counter
import nltk
import torch
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from datasets import Features, ClassLabel

# 1. Загрузка данных и токенизатора
name = "Geotrend/distilbert-base-ru-cased"
tokenizer = AutoTokenizer.from_pretrained(name)

# Загрузка датасета со всеми сплитами
ds = load_dataset("sagteam/cedr_v1", "main")  # Теперь ds содержит 'train' и 'test'

# id2label для справки
id2label = {
    0: "joy",
    1: "sadness",
    2: "surprise",
    3: "fear",
    4: "anger",
    5: "no_emotion"
}
label2id = {v: k for k, v in id2label.items()}


# 3. Очистка текста
nltk.download('stopwords')
ru_stopwords = set(stopwords.words('russian'))


def clean_text(text):
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'@\S+|#\S+', '', text)
    text = re.sub(r'_{5,}', '', text)
    text = re.sub(r'\)+', ')', text)
    text = re.sub(r'\s+', ' ', text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in ru_stopwords and len(t) > 2]
    return ' '.join(tokens) if tokens else None


def process_text(example):
    example['cleaned_text'] = clean_text(example['text'])
    return example


def create_averaged_labels(example):
    # Создаём one-hot вектор с float значениями
    hard_label = [0] * 6
    if example['labels']:
        for label in example['labels']:
            if label < 6:
                hard_label[label] = 1
    if not example['labels']:
        hard_label[5] = 1
    hard_label = torch.tensor(hard_label, dtype=torch.float32)
    # Перезаписываем 'labels' вместо создания нового поля 'label'
    return {
        'label': hard_label
    }


# 5. Обработка каждого сплита
processed = {}
for split in ds:
    temp_ds = ds[split]

    # Очистка текста
    temp_ds = temp_ds.map(process_text)
    temp_ds = temp_ds.filter(lambda x: x['cleaned_text'] is not None)

    # Создание усреднённых меток
    temp_ds = temp_ds.map(create_averaged_labels)

    # Удаление ненужных колонок
    temp_ds = temp_ds.remove_columns(['text', 'source', 'labels'])
    temp_ds = temp_ds.rename_column('label', 'labels')

    processed[split] = temp_ds


# 6. Определение максимальной длины
def get_max_length(dataset, sample_size=1000):
    sample_texts = dataset.shuffle(seed=42).select(range(sample_size))['cleaned_text']
    tokenized = tokenizer(sample_texts, truncation=False, padding=False)
    lengths = [len(x) for x in tokenized['input_ids']]
    return max(lengths)


max_length = get_max_length(processed['train'])
print(max_length)

# 7. Токенизация
def tokenize_function(examples):
    return tokenizer(
        examples['cleaned_text'],
        padding=False,
        truncation=True,
        max_length=128,
        return_special_tokens_mask=True,
        return_attention_mask=True
    )


tokenized_datasets = {}
for split in processed:
    tokenized = processed[split].map(lambda x: tokenize_function(x), batched=True)
    tokenized = tokenized.remove_columns(['cleaned_text'])
    tokenized.set_format(type='torch', columns=['labels','input_ids','attention_mask'])
    tokenized_datasets[split] = tokenized

# 8. Сохранение
tokenized_datasets = DatasetDict(tokenized_datasets)
tokenized_datasets.save_to_disk('./processed_datasets')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Map:   0%|          | 0/7525 [00:00<?, ? examples/s]

Map:   0%|          | 0/1882 [00:00<?, ? examples/s]

77


Map:   0%|          | 0/7525 [00:00<?, ? examples/s]

Map:   0%|          | 0/1882 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7525 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1882 [00:00<?, ? examples/s]

In [18]:
print(tokenized_datasets['test'][0])

{'labels': tensor([0., 0., 0., 0., 0., 1.]), 'input_ids': tensor([   11, 12777,  3167,   175, 10934,  1293,  4590,   326,  4321,  1531,
         3155, 12777,  3167,  9664,   724,   172,  2493,    12]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}


In [27]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler, AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model
from sklearn.metrics import f1_score, accuracy_score
from datasets import load_from_disk
from accelerate import Accelerator
from accelerate.utils import tqdm  # Используем tqdm из accelerate
from timm.layers import LayerNorm2d


# Инициализация Accelerate
accelerator = Accelerator(
    mixed_precision="fp16",
    gradient_accumulation_steps=2
)

# Загрузка модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained("Geotrend/distilbert-base-ru-cased")
model = AutoModelForSequenceClassification.from_pretrained(
    "Geotrend/distilbert-base-ru-cased",
    num_labels=6,
    problem_type="multi_label_classification"
)

# Конфигурация LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"],
    lora_dropout=0.05,
    bias="none",
    modules_to_save=["classifier"],
    use_dora=True
)
model = get_peft_model(model, peft_config)
#print(model)
# Загрузка датасета
tokenized_datasets = tokenized_datasets  # Укажите путь к вашему датасету
train_dataset = tokenized_datasets["test"]  # Используем тестовый набор как тренировочный для примера
eval_dataset = tokenized_datasets["test"]

# Создание DataLoaders
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8
)
eval_dataloader = DataLoader(
    eval_dataset,
    collate_fn=data_collator,
    batch_size=8
)

# Оптимизатор и планировщик
optimizer = AdamW(model.parameters(), lr=2e-4)
num_epochs = 35
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# Подготовка компонентов с помощью Accelerate
model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
)

# Тренировка с tqdm и отображением метрик
model.train()
total_loss = 0
total_accuracy = 0
total_f1 = 0
num_batches = 0

for epoch in range(num_epochs):
    # Оборачиваем тренировочный цикл в tqdm
    progress_bar = tqdm(train_dataloader, 
                       desc=f"Epoch {epoch+1}/{num_epochs}", 
                       disable=not accelerator.is_local_main_process)
    
    for step, batch in enumerate(progress_bar):
        with accelerator.accumulate(model):
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.detach().float()
            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            # Вычисляем метрики для текущего батча
            logits = outputs.logits
            preds = torch.sigmoid(logits) > 0.5
            labels = batch["labels"]
            
            # Собираем данные с всех устройств
            preds_gathered = accelerator.gather(preds)
            labels_gathered = accelerator.gather(labels)
            
            # Вычисляем метрики
            accuracy = accuracy_score(labels_gathered.cpu(), preds_gathered.cpu())
            f1 = f1_score(labels_gathered.cpu(), preds_gathered.cpu(), average="micro")
            
            # Накапливаем метрики
            total_accuracy += accuracy
            total_f1 += f1
            num_batches += 1
            
            # Обновляем прогресс-бар с текущими метриками
            if accelerator.is_local_main_process:
                progress_bar.set_postfix({
                    "loss": loss.item(),
                    "accuracy": f"{accuracy:.4f}",
                    "f1": f"{f1:.4f}"
                })

    # Вычисляем средние метрики за эпоху
    avg_loss = total_loss / len(train_dataloader)
    avg_accuracy = total_accuracy / num_batches
    avg_f1 = total_f1 / num_batches
    
    # Выводим итоговые метрики за эпоху
    if accelerator.is_local_main_process:
        print(f"\nEpoch {epoch+1} completed:")
        print(f"Average Loss: {avg_loss:.4f}")
        print(f"Average Accuracy: {avg_accuracy:.4f}")
        print(f"Average F1: {avg_f1:.4f}\n")

# Оценка
model.eval()
all_preds, all_labels = [], []

for batch in eval_dataloader:
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    preds = torch.sigmoid(logits) > 0.5
    all_preds.append(accelerator.gather(preds))
    all_labels.append(accelerator.gather(batch["labels"]))

# Объединение результатов
all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)
all_preds_np = all_preds.cpu().numpy()
all_labels_np = all_labels.cpu().numpy()

# Вычисление метрик
f1 = f1_score(all_labels_np, all_preds_np, average="micro")
accuracy = accuracy_score(all_labels_np, all_preds_np)

print(f"Evaluation results - F1: {f1:.4f}, Accuracy: {accuracy:.4f}")
accelerator.wait_for_everyone()

# Сохраните модель (только LoRA-адаптеры и классификатор)
output_dir = "./lora_classifier_model"
model.save_pretrained(output_dir)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at Geotrend/distilbert-base-ru-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/35:   0%|          | 0/236 [00:00<?, ?it/s]


Epoch 1 completed:
Average Loss: 0.3810
Average Accuracy: 0.2399
Average F1: 0.3180



Epoch 2/35:   0%|          | 0/236 [00:00<?, ?it/s]


Epoch 2 completed:
Average Loss: 0.6918
Average Accuracy: 0.3480
Average F1: 0.4403



...


Epoch 33 completed:
Average Loss: 3.8398
Average Accuracy: 0.7910
Average F1: 0.8442



Epoch 34/35:   0%|          | 0/236 [00:00<?, ?it/s]


Epoch 34 completed:
Average Loss: 3.8600
Average Accuracy: 0.7962
Average F1: 0.8483



Epoch 35/35:   0%|          | 0/236 [00:00<?, ?it/s]


Epoch 35 completed:
Average Loss: 3.8790
Average Accuracy: 0.8012
Average F1: 0.8521

Evaluation results - F1: 0.9997, Accuracy: 0.9995
